In [ ]:
import sys

In [ ]:
sys.path.append('..')

In [ ]:
import mc_api_utils

In [ ]:
from config import BOT_TOKEN, login, password

In [ ]:
token = mc_api_utils.autorization(login, password)

In [ ]:
url = 'https://mc.dev.rand.agency/page/45364153'

In [ ]:
# code, pages = mc_api_utils.get_my_pages(token)

In [ ]:
code, biography = mc_api_utils.get_biography(url, token)

In [ ]:
images_num = 0
for i in biography:
    try:
        images_num+=len(biography[i]['images'])
    except Exception as ex:
        pass
images_num

In [ ]:
biography

In [ ]:
code, img = mc_api_utils.load_photo('https://src.mc.dev.rand.agency/storage/app/public/11017/1732697462.jpg')

In [ ]:
img

# Discript image

In [ ]:

from transformers import AutoModelForVision2Seq, BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForVision2Seq.from_pretrained(
    "HuggingFaceTB/SmolVLM-Instruct",
    quantization_config=quantization_config,
)

In [ ]:
from transformers import AutoProcessor, AutoModelForVision2Seq
import torch

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-Instruct")

model = AutoModelForVision2Seq.from_pretrained(
    "HuggingFaceTB/SmolVLM-Instruct",
    torch_dtype=torch.bfloat16
).to("cuda")

In [ ]:
from PIL import Image
from transformers.image_utils import load_image


# Load images
image1 = img.copy()

# Create input messages
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "Can you describe the image?"} #Please write a good short script that is well suited for creating an animation of this picture?
        ]
    },
]

# Prepare inputs
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[image1], return_tensors="pt")
inputs = inputs.to(DEVICE)

In [ ]:
# Generate outputs
generated_ids = model.generate(**inputs, max_new_tokens=77)
generated_texts = processor.batch_decode(
    generated_ids,
    skip_special_tokens=True,
)

print(generated_texts[0])

In [ ]:
prompt = generated_texts[0].split('Assistant:')[1]

In [ ]:
prompt

# Generate video

# PAI

In [ ]:
import torch
from diffusers import (
    EulerDiscreteScheduler,
    MotionAdapter,
    PIAPipeline,
)
from diffusers.utils import export_to_gif, load_image

adapter = MotionAdapter.from_pretrained("openmmlab/PIA-condition-adapter")
pipe = PIAPipeline.from_pretrained("SG161222/Realistic_Vision_V6.0_B1_noVAE", motion_adapter=adapter, torch_dtype=torch.float16)

pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()
pipe.enable_vae_slicing()



In [ ]:
image = img.copy().resize((512, 512))
# prompt = "A car realystic driving"
prompt = "The man is holding a helmet in his left hand and raising his right arm."
negative_prompt = "wrong white balance, dark, sketches,worst quality,low quality"

generator = torch.Generator("cpu").manual_seed(0)
output = pipe(image=image, prompt=prompt, generator=generator)
frames = output.frames[0]
export_to_gif(frames, "pia-animationm.gif")

##

## ModelscopeT2V

In [ ]:
import torch
from diffusers import StableVideoDiffusionPipeline
from diffusers.utils import load_image, export_to_video

pipe = StableVideoDiffusionPipeline.from_pretrained(
    "stabilityai/stable-video-diffusion-img2vid", torch_dtype=torch.float16, variant="fp16"
)
pipe.enable_model_cpu_offload()

image = img.copy()

generator = torch.manual_seed(42)
frames = pipe(image, decode_chunk_size=8, generator=generator, num_frames=25).frames[0]
export_to_video(frames, "generatedm.mp4", fps=7)

## I2VGen-XL

In [ ]:
import torch
from diffusers import I2VGenXLPipeline
from diffusers.utils import export_to_gif, load_image

pipe = I2VGenXLPipeline.from_pretrained("ali-vilab/i2vgen-xl", torch_dtype=torch.float16, variant="fp16")
pipe.enable_model_cpu_offload()




In [ ]:
# image_url = "https://huggingface.co/datasets/diffusers/docs-images/resolve/main/i2vgen_xl_images/img_0009.png"
image = img.copy()

prompt = prompt #"The man is holding a helmet in his left hand and raising his right arm."
negative_prompt = "Distorted, discontinuous, Ugly, blurry, low resolution, motionless, static, disfigured, disconnected limbs, Ugly faces, incomplete arms"
generator = torch.manual_seed(8888)
frames = pipe(
    prompt=prompt,
    image=image,
    num_inference_steps=50,
    negative_prompt=negative_prompt,
    guidance_scale=9.0,
    generator=generator
).frames[0]
export_to_gif(frames, "i2vm.gif")

In [ ]:
from diffusers.utils import load_image, export_to_video
export_to_video(frames, "generatedm.mp4", fps=7)

In [ ]:
import tempfile

In [ ]:
def export_to_gif(image, output_gif_path: str = None, fps: int = 10) -> str:
    if output_gif_path is None:
        output_gif_path = tempfile.NamedTemporaryFile(suffix=".gif").name

    image[0].save(
        output_gif_path,
        save_all=True,
        append_images=image[1:],
        optimize=False,
        duration=1000 // fps,
        loop=0,
    )
    return output_gif_path

In [ ]:
gif = export_to_gif(frames, "generatedm.gif", fps=10)

In [ ]:
gif

In [ ]:
export_to_gif(frames, "i2vm.gif")

In [ ]:
import cv2

In [ ]:
videodims = frames[0].size
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
FPS = 30
video = cv2.VideoWriter('output.mp4', fourcc, FPS, videodims)

In [ ]:
for fidx, f in enumerate(data):
    print( f'Done: {round(fidx * 100 / len(data), 1)} % - {f}', end="\r")
    img = Image.open(f)
    video.write(cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR))
video.release()

## COgVideoX

In [ ]:
import torch
from diffusers import CogVideoXImageToVideoPipeline
from diffusers.utils import export_to_video, load_image

prompt = "The man is holding a helmet in his left hand and raising his right arm."
image = img.copy()
pipe = CogVideoXImageToVideoPipeline.from_pretrained(
    "THUDM/CogVideoX-5b-I2V",
    torch_dtype=torch.bfloat16
)
 


In [ ]:
pipe.vae.enable_tiling()
pipe.vae.enable_slicing()

video = pipe(
    prompt=prompt,
    image=image,
    num_videos_per_prompt=1,
    num_inference_steps=50,
    num_frames=49,
    guidance_scale=6,
    generator=torch.Generator(device="cuda").manual_seed(42),
).frames[0]

export_to_video(video, "output.mp4", fps=8)